[View in Colaboratory](https://colab.research.google.com/github/Bewica/challenge/blob/master/boss_or_not.ipynb)

In [0]:
import pandas as pd
import requests

In [2]:
emails = pd.read_csv('https://storage.googleapis.com/bewica-challenge/emails_tenth.csv', engine='python', error_bad_lines=False)


Skipping line 18226: field larger than field limit (131072)
Skipping line 20082: ',' expected after '"'
Skipping line 30906: field larger than field limit (131072)
Skipping line 31519: field larger than field limit (131072)
Skipping line 32130: field larger than field limit (131072)
Skipping line 37541: field larger than field limit (131072)
Skipping line 37688: ',' expected after '"'
Skipping line 37725: ',' expected after '"'
Skipping line 37952: ',' expected after '"'
Skipping line 38004: field larger than field limit (131072)
Skipping line 38008: ',' expected after '"'
Skipping line 38011: ',' expected after '"'
Skipping line 38068: ',' expected after '"'
Skipping line 38076: ',' expected after '"'
Skipping line 38083: ',' expected after '"'
Skipping line 38085: ',' expected after '"'
Skipping line 38086: ',' expected after '"'
Skipping line 38090: ',' expected after '"'
Skipping line 38101: ',' expected after '"'
Skipping line 38108: ',' expected after '"'
Skipping line 38145: ','

In [0]:
def extract_fields(message):
  email_from = ""
  email_to = ""
  date = ""
  subject = ""
  xfrom = ""
  xto = ""
  body = ""
  prevline = ""
  start_body = False
  lines = str(message).split("\n")
  if len(lines) > 0:
    for line in lines:
      try:
        if line.startswith("From:"):
          splitline = line.split("From: ")
          if len(splitline) > 1:
            email_from = line.split("From: ")[1]
        if line.startswith("To:"):
          splitline = line.split("To: ")
          if len(splitline) > 1:
            email_to = line.split("To: ")
        if line.startswith("Date:"):
          splitline = line.split("Date: ")
          if len(splitline) > 1:
            date = line.split("Date: ")[1]
        if line.startswith("Subject:"):
          splitline = line.split("Subject: ")
          if len(splitline) > 1:
            subject = line.split("Subject: ")[1]
        if line.startswith("X-From:"):
          splitline = line.split("X-From: ")
          if len(splitline) > 1:
            xfrom = line.split("X-From: ")[1]
        if line.startswith("X-To:"):
          splitline = line.split("X-To: ")
          if len(splitline) > 1:
            xto = line.split("X-To: ")[1]
        if (prevline.startswith("X-") or start_body) and not line.startswith("X-"):
          body += line
          start_body = True
        prevline = line
      except:
        pass
  return pd.Series([email_from,email_to,xfrom,xto,date,subject,body])

In [0]:
emails['user'] = emails.apply(lambda email: str(email.file).split('/')[0], axis=1)
def take_second(text):
  if len(str(text).split('/')) > 1:
    return str(text).split('/')[1]
emails['type'] = emails.apply(lambda email: take_second(email.file), axis=1)
emails[['email_from','email_to','xfrom','xto','date','subject','body']] = emails.apply(lambda email: extract_fields(email.message), axis=1)

In [0]:
def has_boss_name(name):
  name = name.lower()
  if ("ken lay" in name) or ("ken.lay" in name) or ("kenneth lay" in name):
    return True
  else:
    return False

emails['from_boss'] = emails.apply(lambda email: has_boss_name(email.email_from) or has_boss_name(email.xfrom), axis=1)

In [11]:
emails[emails.from_boss].body.tail(10)

59339    Okay, I've already had questions about stateme...
59340    FYI - Thought you might be interested in Gary ...
59905    Today, Enron hosted a conference call to give ...
59926    Today we announced another positive developmen...
60063    As you have heard, Dynegy is terminating the m...
60131    Many of you have asked whether you should come...
61143    I want to remind you about our All-Employee Me...
61150    Today we announced the appointment of Jeff McM...
61342    I know that this is a difficult time for all o...
61343    Today we released additional information about...
Name: body, dtype: object